### install from https://github.com/amish-mishra/cechmate-DR/tree/master

In [1]:
import cechmate as cm
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from persim.persistent_entropy import *
import gudhi.representations
import gudhi as gd 
from utils import *

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

housing = fetch_california_housing()

X_all = pd.DataFrame(housing.data, columns = housing.feature_names)
y_all = housing['target']

X_all = X_all.iloc[:1000]
y_all = y_all[:1000]

In [3]:
scaler = StandardScaler()
scaler.fit(X_all)
X_scaled = pd.DataFrame(scaler.transform(X_all), index= X_all.index, columns= X_all.columns)
X_scaled.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,2.475751,0.266201,1.500473,-0.266636,-0.900449,-0.219674,1.441017,-0.640477
1,2.462605,-1.251479,0.863503,-0.751121,1.186500,-0.830714,1.235934,-0.554449
2,1.885951,1.100925,2.613913,0.196463,-0.725783,0.118538,1.133393,-0.726504
3,0.994291,1.100925,0.504211,0.192853,-0.663546,-0.230108,1.133393,-0.812532
4,0.001771,1.100925,0.900830,0.267694,-0.656520,-0.732521,1.133393,-0.812532


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_all, test_size=0.5, random_state=43)

lr = LinearRegression()
lr.fit(X_train, y_train)
predictions_lr = lr.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions_lr))
r2 = r2_score(y_test, predictions_lr)

print('RMSE:', rmse)
print('R-square:', r2)

RMSE: 0.5137944706615996
R-square: 0.6499318732684514


## Topo Feature

In [5]:
data_list = Data_list(X_scaled)
len(data_list)

Processing row 500 out of 1000
Processing row 1000 out of 1000


1000

### Alpha from Cechmate

In [8]:
# results = Parrallel_compute(data_list, typeComplex='AlphafromCechmate')

## Delaunay-Rips

In [6]:
results = Parrallel_compute(data_list, typeComplex='DR')

平行運算時間:  35.906028032302856


## Alpha

In [9]:
results = Parrallel_compute(data_list, typeComplex='Alpha')

平行運算時間:  9.03106689453125


In [13]:
import pickle
# with open('Calculated/results.pkl', 'wb') as f:
#     pickle.dump(results, f)

with open('Calculated/results.pkl', 'rb') as file:
    results = pickle.load(file)

In [10]:
X_scaled['topo_feature'] = [x.tolist()[0][0] if isinstance(x, np.ndarray) else x for x in results]
X_scaled['topo_feature'] = scaler.fit_transform(X_scaled[['topo_feature']])
X_scaled.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,topo_feature
0,2.475751,0.266201,1.500473,-0.266636,-0.900449,-0.219674,1.441017,-0.640477,0.130263
1,2.462605,-1.251479,0.863503,-0.751121,1.186500,-0.830714,1.235934,-0.554449,0.455036
2,1.885951,1.100925,2.613913,0.196463,-0.725783,0.118538,1.133393,-0.726504,-0.523892
3,0.994291,1.100925,0.504211,0.192853,-0.663546,-0.230108,1.133393,-0.812532,-0.288214
4,0.001771,1.100925,0.900830,0.267694,-0.656520,-0.732521,1.133393,-0.812532,-0.084631


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_all, test_size=0.5, random_state=42)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = pd.Series(y_train).reset_index(drop=True)
y_test = pd.Series(y_test).reset_index(drop=True)

lr = LinearRegression()
lr.fit(X_train, y_train)
predictions_lr = lr.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, predictions_lr))
r2 = r2_score(y_test, predictions_lr)

print('RMSE:', rmse)
print('R-square:', r2)

RMSE: 0.5301842830951464
R-square: 0.6487234845943991
